<a href="https://colab.research.google.com/github/linlih/ML_By_NTU_Hung-yi-Lee/blob/master/ml_2019spring_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作业介绍


作业地址：

https://ntumlta2019.github.io/ml-web-hw2/

投影片：

https://docs.google.com/presentation/d/1f4a81OqeG27pRHuqHXg7Qi0KJ4SlLBDpe9ljNu3IXT0/edit#slide=id.p19

Kaggle链接：

https://www.kaggle.com/c/ml2019spring-hw2

作业说明：

需要实现的根据一个人的数据信息来预测这个人的年收入是否大于50K

共有32561笔训练资料，16281笔测试资料


# 数据下载

In [0]:
from google.colab import files
files.upload()

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [9]:
!kaggle competitions download -c ml2019spring-hw2

  0% 0.00/468k [00:00<?, ?B/s]
100% 468k/468k [00:00<00:00, 69.4MB/s]
  0% 0.00/224k [00:00<?, ?B/s]
100% 224k/224k [00:00<00:00, 73.5MB/s]
  0% 0.00/298k [00:00<?, ?B/s]
100% 298k/298k [00:00<00:00, 92.0MB/s]
  0% 0.00/116k [00:00<?, ?B/s]
100% 116k/116k [00:00<00:00, 119MB/s]
  0% 0.00/595k [00:00<?, ?B/s]
100% 595k/595k [00:00<00:00, 82.5MB/s]
  0% 0.00/63.6k [00:00<?, ?B/s]
100% 63.6k/63.6k [00:00<00:00, 64.6MB/s]


In [10]:
!unzip test.csv.zip
!unzip train.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


# 数据处理

因为特征中包含了缺失值，也包含了非类别的内容，我们要做一下处理

首先需要区分下数值特征(6个)和类别特征(8个)：

**continuous**: age, fnlwgt, education_num, capital_gain, capital_loss, hours_per_week

**categorical**: workclass, education, marital_status, occupation, relationship, race, sex, native_country

train.csv : 32561 $\times$ 15

test.csv : 16281 $\times$ 14

测试数据多处理来的一列是收入income，也就是我们要预测的那个结果

本质这个问题就是一个二分类问题

要处理的问题如下：

1. 数值类型的特征需要中心化
2. 类别类型的特征需要OneHot编码

In [0]:
import numpy as np
import pandas as pd

In [0]:
data = pd.read_csv('train.csv',header=1)

In [0]:
data.hist(bins=50, figsize=(10, 8))

In [0]:
from sklearn.impute import SimpleImputer
data = pd.read_csv('train.csv')

In [0]:
train_num_col = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
train_cat_col = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
train_num = data[train_num_col]
train_cat = data[train_cat_col]

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(data[['occupation']])
cat_encoder = OneHotEncoder()
cat_encoder.fit_transform(housing_cat_encoded).toarray()

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), # 缺失值填写为中位数
        ('std_scaler', StandardScaler()),
    ])

#train_num_tr = num_pipeline.fit_transform(train_num)

In [33]:
train_num_tr.shape

(32561, 6)

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")), # 缺失值填写为中位数
        ('ord_encoder', OrdinalEncoder()),
        ('one_hot', OneHotEncoder()),
    ])

#train_cat_tr = cat_pipeline.fit_transform(train_cat)

In [0]:
data.head(10)

In [0]:
from sklearn.compose import ColumnTransformer
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, train_num_col),
        ("cat", cat_pipeline, train_cat_col),
    ])
data_t = data.drop(['income'], axis=1)
train_prepared = full_pipeline.fit_transform(data_t)

In [8]:
train_prepared.shape

(32561, 108)

In [0]:
y = np.array(pd.Categorical(data['income']).codes) 

In [0]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test = train_test_split(train_prepared, y, train_size=0.7)

In [37]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22792 entries, 13290 to 15610
Columns: 102 entries, workclass_ ? to native_country_ Yugoslavia
dtypes: uint8(102)
memory usage: 3.0 MB


In [11]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(Cs=np.logspace(-3, 4, 8), cv=5)
clf.fit(x, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegressionCV(Cs=array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04]),
                     class_weight=None, cv=5, dual=False, fit_intercept=True,
                     intercept_scaling=1.0, l1_ratios=None, max_iter=100,
                     multi_class='auto', n_jobs=None, penalty='l2',
                     random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [12]:
from sklearn import metrics
y_hat = clf.predict(x)
print('训练集精确度：', metrics.accuracy_score(y, y_hat))

训练集精确度： 0.8545981045981046


In [13]:
y_test_hat = clf.predict(x_test)
print('测试集精确度：', metrics.accuracy_score(y_test, y_test_hat))

测试集精确度： 0.8487050875217524


# 测试

In [14]:
import pandas
test_data = pandas.read_csv('test.csv', encoding='utf-8')# 测试文件的第一行不是列属性
test_data.shape

(16281, 14)

In [24]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy="most_frequent")
print(imp.fit_transform(test_data))

[[25 ' Private' 226802 ... 0 40 ' United-States']
 [38 ' Private' 89814 ... 0 50 ' United-States']
 [28 ' Local-gov' 336951 ... 0 40 ' United-States']
 ...
 [38 ' Private' 374983 ... 0 50 ' United-States']
 [44 ' Private' 83891 ... 0 40 ' United-States']
 [35 ' Self-emp-inc' 182148 ... 0 60 ' United-States']]


In [0]:
test_prepared = full_pipeline.transform(test_data)

In [0]:
y_test_hat = clf.predict(test_prepared)

In [18]:
y_test_hat.shape

(16281,)

# 提交

In [0]:
# 包含小数部分的提交

results = pd.Series(y_test_hat, name="label")
submission = pd.concat([pd.Series(["%d" % i for i in range(1,y_test_hat.shape[0] + 1)],name = "id"),results],axis = 1)
submission.to_csv('submission.csv', index=False)

In [24]:
!kaggle competitions submit ml2019spring-hw2 -f submission.csv -m "submit from colab"

100% 116k/116k [00:02<00:00, 48.2kB/s]
Successfully submitted to ML2019spring-hw2